In [11]:
import boto3
import os
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm

client = boto3.client('s3')

bucket = "ml-for-bem"
# experiment_name = "single-climate-zone/v1/nyc"
# experiment_name = "full_climate_zone/v1/train"
# experiment_name = "full_climate_zone/v2/test"
# experiment_name = "full_climate_zone/v3/train"
# experiment_name = "full_climate_zone/v3/test"
# experiment_name = "full_climate_zone/v4/train"
experiment_name = "full_climate_zone/v4/test"

In [12]:
paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket, Prefix=experiment_name + "/errors/")

error_ids = []
files = []
for page in pages:
    for obj in page['Contents']:
        files.append(obj['Key'])
        error_ids.append(obj['Key'].split('/')[-1].split('.')[0])
print(len(error_ids))

KeyError: 'Contents'

In [13]:
# get a list of all the files in the /monthly/ folder

pages = paginator.paginate(Bucket=bucket, Prefix=experiment_name + "/monthly/")

monthly_files = []
for page in pages:
    for obj in page['Contents']:
        monthly_files.append(obj['Key'])

In [14]:
dfs = []

# download all of the files in the monthly folder to data/{experiment_name}
os.makedirs("data/hdf5/" + experiment_name + "/monthly", exist_ok=True)
i = 0
for file in tqdm(monthly_files):
    client.download_file(bucket, file, "data/hdf5/" + file)
    with pd.HDFStore("data/hdf5/" + file) as store:
        dfs.append(store["batch_results"])
    i = i + 1
    os.remove("data/hdf5/" + file)

dfs = pd.concat(dfs, axis=0)
print(len(dfs))

  0%|          | 0/1992 [00:00<?, ?it/s]

39009


In [15]:
ids = dfs.index.get_level_values("id")    
error_mask = ids.isin(error_ids)
dfs = dfs.set_index(error_mask,  append=True, )
dfs.index.names = dfs.index.names[:-1] + ["error"]
dfs.index.to_frame(index=False).head()
dfs.to_hdf("data/hdf5/" + experiment_name + "/monthly.hdf", key="batch_results", mode="w")

In [16]:
# upload the monthly.hdf file to s3
client.upload_file("data/hdf5/" + experiment_name + "/monthly.hdf", bucket, experiment_name + "/monthly.hdf")